# Contents

* [<font size=4>Part1. Tensorflow basics</font>](#1)
    * [Tensors](#1.1)
    * [Random Constant tensors](#1.2)
    * [Variables](#1.3)
    * [Doing math in tensorflow](#1.4)
    * [Computing gradients with gradienttape](#1.5)
   

In [1]:
! pip install tensorflow

     -------------------------------------- 455.9/455.9 MB 2.3 MB/s eta 0:00:00
     -------------------------------------- 438.7/438.7 kB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 14.2/14.2 MB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 124.6/124.6 kB ? eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 8.2 MB/s eta 0:00:00
     -------------------------------------- 895.9/895.9 kB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.3 MB/s eta 0:00:00
     -------------------------------------- 175.4/175

In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.0


# Part1. Tensorflow basics <a id="1"></a>

## Tensors <a id="1.1"></a>
This is a [constant tensor](https://www.tensorflow.org/api_docs/python/tf/constant)

In [3]:
x = tf.constant([[5,2], [1,3]])
print(x)

tf.Tensor(
[[5 2]
 [1 3]], shape=(2, 2), dtype=int32)


You can get its value as a Numpy array by calling `.numpy():`

In [4]:
x.numpy()

array([[5, 2],
       [1, 3]])

Much like a numpy array, it features the attributes `dtype` and `shape`:

In [5]:
print('dtype:', x.dtype)
print('shape:', x.shape)

dtype: <dtype: 'int32'>
shape: (2, 2)


A common way to create constant tensors is via `tf.ones` and `tf.zeros` (just like `np.ones` and `np.zeros`):

In [6]:
print(tf.ones(shape=(2,1)))
print(tf.zeros(shape=(2,1)))

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


## Random constant tensors <a id="1.2"></a>


This is all pretty [normal](https://www.tensorflow.org/api_docs/python/tf/random/normal):

In [7]:
tf.random.normal(shape=(2,2), mean=0., stddev=1.)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 2.1929123 , -0.6579585 ],
       [-0.6433016 ,  0.89284545]], dtype=float32)>

And here's an integer tensor with values drawn from a random [uniform](https://www.tensorflow.org/api_docs/python/tf/random/uniform) distribution:

In [8]:
tf.random.uniform(shape=(2, 2), minval=0, maxval=10, dtype='int32')

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 5],
       [9, 6]])>

## Variables <a id="1.3"></a>

Variables are special tensors used to store mutable state (like the weights of a neural network). You create a Variable using some initial value.

In [9]:
initial_val = tf.random.normal(shape=(2,2))
a = tf.Variable(initial_val)
print(a)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[-0.72060364, -0.32833654],
       [-0.2150494 , -1.7650609 ]], dtype=float32)>


You update the value of a Variable by using the methods `.assign(value)`, or `.assign_add(increment)` or `.assign_sub(decrement)`:

In [10]:
new_value = tf.random.normal(shape=(2,2))
a.assign(new_value)
for i in range(2):
    for j in range(2):
        assert a[i, j]== new_value[i,j]

In [11]:
add_value = tf.random.normal(shape=(2,2))
a.assign_add(add_value)
for i in range(2):
    for j in range(2):
        assert a[i,j] == new_value[i,j] + add_value[i,j]

## Doing math in TensorFlow <a id="1.4"></a>

You can use TensorFlow exactly like you would use Numpy. The main difference is that your TensorFlow code can run on GPU and TPU.

In [13]:
a = tf.random.normal(shape=(2,2))
b = tf.random.normal(shape=(2,2))

c = a + b
print(c)
d = tf.square(c)
print(d)


tf.Tensor(
[[ 0.17592186 -0.11836635]
 [ 1.6019094  -0.69676244]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0.0309485  0.01401059]
 [2.5661137  0.4854779 ]], shape=(2, 2), dtype=float32)


## Computing gradients with GradientTape <a id="1.5"></a>

there's another big difference with Numpy: you can automatically retrieve the gradient of any differentiable expression.

Just open a GradientTape, start "watching" a tensor via `tape.watch()`, and compose a differentiable expression using this tensor as input:


In [14]:
a = tf.random.normal(shape=(2,2))
b = tf.random.normal(shape=(2,2))

with tf.GradientTape() as tape:
    tape.watch(a)
    c = tf.sqrt(tf.square(a) + tf.square(b))
    dc_da = tape.gradient(c, a)
    print(dc_da)

tf.Tensor(
[[ 0.8598611  -0.5558522 ]
 [-0.46526164  0.95366484]], shape=(2, 2), dtype=float32)


By default, variables are watched automatically, so you don't need to manually `watch` them:

In [15]:
a = tf.Variable(a)
with tf.GradientTape() as tape:
    c = tf.sqrt(tf.square(a) + tf.square(b))
    dc_da = tape.gradient(c,a)
    print(dc_da)

tf.Tensor(
[[ 0.8598611  -0.5558522 ]
 [-0.46526164  0.95366484]], shape=(2, 2), dtype=float32)


Note that you can compute higher-order derivatives by nesting tapes:

In [ ]:
with tf.GradientTape() as outer_tape:
  with tf.GradientTape() as tape:
    c = tf.sqrt(tf.square(a) + tf.square(b))/'
    dc_da = tape.gradient(c, a)
  d2c_da2 = outer_tape.gradient(dc_da, a)
  print(d2c_da2)

tf.Tensor(
[[0.22945738 2.5800362 ]
 [0.28621    0.04524323]], shape=(2, 2), dtype=float32)
